# Preprocessing: LL13


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


Data was actually recleaned using only the regions of interest because it was so messy. Annotations were resaved and then the cleaning script was rerun/

In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
# %matplotlib qt5

In [2]:
## Prep paths ##

subject = 'LL13'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [14]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/LL13/ieeg/LL13_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/LL13/ieeg/LL13_raw_clean_ieeg.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.


In [15]:
## Channels did not include probe prefixes in bci data file, convert to descriptive names here

# read in csv with correct names
channel_csv = pd.read_csv(f"{preproc_data_dir}/../LL13_correct_channel_names.csv")

# convert to dict
channel_dict = dict(zip(channel_csv.bi_names, channel_csv.use_names))

# rename
mne.rename_channels(filtered_clean_fif.info, channel_dict)
mne.rename_channels(raw_clean_fif.info, channel_dict)


## Bipolar Rereferencing

In [16]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe and probe is not None]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [17]:
raw_clean_fif.info['bads'].extend(['EKG'])
filtered_clean_fif.info['bads'].extend(['EKG'])

In [18]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
# %debug
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

LA1 noref
LA2 noref
LA3 noref
LA4 noref
LA5 noref
LA6 noref
LA7 noref
LA8 noref
LH1 noref
LH2 noref
LH3 LH4
LH4 LH5
LH5 LH7
LH6 noref
LH7 LH8
LHP1 LHP2
LHP2 LHP3
LHP3 LHP4
LHP4 LHP5
LHP5 LHP6
LHP6 LHP7
LHP7 LHP8
LOF1 LOF2
LOF2 LOF3
LOF3 LOF4
LOF4 LOF5
LOF5 LOF6
LOF6 LOF7
LOF7 LOF8
LOF8 LOF9
LOF9 LOF10
LAC1 LAC2
LAC2 LAC3
LAC3 LAC4
LAC4 LAC5
LAC5 LAC6
LAC6 LAC7
LAC7 LAC8
RA1 RA2
RA2 RA3
RA3 RA4
RA4 RA5
RA5 RA8
RA6 noref
RA7 noref
RH1 RH2
RH2 RH3
RH3 RH6
RH4 noref
RH5 noref
RH6 RH7
RH7 RH8
RHP1 RHP2
RHP2 RHP3
RHP3 RHP4
RHP4 RHP5
RHP5 RHP6
RHP6 RHP7
RHP8 noref
ROF1 ROF2
ROF2 ROF3
ROF3 ROF4
ROF4 ROF5
ROF5 ROF6
ROF6 ROF7
ROF7 ROF8
ROF8 ROF9
ROF9 ROF10
ROF10 ROF11
ROF11 ROF12
RAC1 RAC2
RAC2 RAC3
RAC3 RAC4
RAC4 RAC5
RAC5 RAC6
RAC6 RAC7
RAC7 RAC8
RAC8 RAC9
RAC9 RAC10
EKG noref
CH89 noref
CH90 noref
CH91 noref
CH92 noref
CH93 noref
CH94 noref
CH95 noref
CH96 noref
CH97 noref
CH98 noref
CH99 noref
CH100 noref
CH101 noref
CH102 noref
CH103 noref
CH104 noref
CH105 noref
CH106 noref
CH107 noref
CH1

In [19]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 901099  =      0.000 ...  1759.959 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=63, n_times=901100
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
Added the following bipolar channels:
LH3-LH4, LH4-LH5, LH5-LH7, LH7-LH8, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LHP6-LHP7, LHP7-LHP8, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA8, RH1-RH2, RH2-RH3, RH3-RH6, RH6-RH7, RH7-RH8, RHP1-RHP2, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, ROF11-ROF12, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7, RAC7-RAC8, RAC8-RAC9, RAC9-RAC10
Overwriting existing file.
Writing /home/brooke/pacman/pr

/tmp/ipykernel_30104/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif
[done]
Reading 0 ... 901099  =      0.000 ...  1759.959 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=63, n_times=901100
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
Added the following bipolar channels:
LH3-LH4, LH4-LH5, LH5-LH7, LH7-LH8, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LHP6-LHP7, LHP7-LHP8, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA8, RH1-RH2, RH2-RH3, RH3-RH6, RH6-RH7, RH7-RH8, RHP1-RHP2, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, ROF11-ROF12, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7,

/tmp/ipykernel_30104/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_raw_clean_data.fif
[done]


In [61]:
# Visualize it #
bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")
events = mne.find_events(bp_raw_fif, output='step', consecutive = False, stim_channel='STI')
bp_raw_fif.pick_channels(['RH1-RH2', 'LOF1-LOF2', 'ROF1-ROF2', 'RA1-RA2', 'RA2-RA3', 'LAC1-LAC2', 'LAC2-LAC3', 'RAC1-RAC2', 'LOF7-LOF8',
 'LOF8-LOF9',
 'LOF9-LOF10',
 'ROF8-ROF9',
 'ROF9-ROF10',
 'ROF10-ROF11',
 'ROF11-ROF12',
 'RAC6-RAC7',
 'RAC7-RAC8',
 'RAC8-RAC9',
 'RAC9-RAC10'])
bp_raw_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 30, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_raw_clean_data.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.


/tmp/ipykernel_30104/4246911768.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")


480 events found
Event IDs: [0 1]


<MNEBrowseFigure size 1846x1016 with 4 Axes>

In [9]:
# # From interactive sessions
# interactive_annot = bp_raw_fif.annotations
# for x in range(0, len(bp_raw_fif.annotations)):
#     print(interactive_annot[x])
    
    
# bp_raw_fif.annotations.save(f'{raw_data_dir}/ieeg/saved_annotations.fif', overwrite = True)    

OrderedDict([('onset', 0.313664), ('duration', 2.4407012462615967), ('description', 'BAD_'), ('orig_time', None)])
OrderedDict([('onset', 79.681641), ('duration', 1.9921875), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 82.730469), ('duration', 7.990234375), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 91.78125), ('duration', 2.44921875), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 109.519531), ('duration', 2.10546875), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 179.779297), ('duration', 8.501953125), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 208.423828), ('duration', 1.947265625), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 276.136719), ('duration', 1.9921875), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 281.4043485449735), ('duration', 1.1760085978835946), ('description', 'BAD_'), ('orig_time', None)])
OrderedD

/tmp/ipykernel_30104/3654251124.py:7: RuntimeWarning: This filename (/home/brooke/pacman/raw_data/LL13/ieeg/saved_annotations.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  bp_raw_fif.annotations.save(f'{raw_data_dir}/ieeg/saved_annotations.fif', overwrite = True)


## Epoching the data 

### Onset

In [5]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
  
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)

# # save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.


/tmp/ipykernel_233518/1846923726.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 8193 original time points ...
Loading data for 240 events and 8193 original time points ...


/tmp/ipykernel_233518/1846923726.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Trial End

In [6]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1]
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
480 events found
Event IDs: [0 1]


/tmp/ipykernel_233518/1129154783.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 239 events and 5121 original time points ...


/tmp/ipykernel_233518/1129154783.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Last Away

In [7]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,3,11,39347,0,1
1,4,8,41523,0,1
2,5,16,46054,0,1
3,6,10,47795,0,1
4,7,3,51993,0,1
...,...,...,...,...,...
220,234,18,851404,0,1
221,235,19,855321,0,1
222,236,20,858393,0,1
223,237,9,863104,0,1


In [8]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [9]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
Not setting metadata
225 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 5121 original time points ...
Loading data for 225 events and 5121 original time points ...


/tmp/ipykernel_233518/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_233518/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [10]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,7,25241,0,1
1,1,15,28672,0,1
2,2,1,33331,0,1
3,3,11,39142,0,1
4,7,3,51916,0,1
...,...,...,...,...,...
216,234,18,850790,0,1
217,235,19,854732,0,1
218,236,20,857804,0,1
219,237,9,862848,0,1


In [11]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [12]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
Not setting metadata
221 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 4609 original time points ...
Loading data for 221 events and 4609 original time points ...


/tmp/ipykernel_233518/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_233518/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [13]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,7,25113,0,1
1,1,15,28492,0,1
2,2,1,33203,0,1
3,3,11,39116,0,1
4,4,8,41523,0,1
...,...,...,...,...,...
234,234,18,850764,0,1
235,235,19,854579,0,1
236,236,20,857702,0,1
237,237,9,862745,0,1


In [14]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [15]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif...
    Range : 0 ... 901099 =      0.000 ...  1759.959 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 5121 original time points ...
Loading data for 239 events and 5121 original time points ...


/tmp/ipykernel_233518/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_233518/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL13/ieeg/LL13_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')
